In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
import time
import tensorflow as tf
import math

# train
import torch
from torch import nn
from torch.nn import functional as F

# load data
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## 1. Data
* you can use any data normalisation method
* one example of the data normalisation is whitenning as given by:

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,)),  # mean value = 0.1307, standard deviation value = 0.3081
])

* load the MNIST dataset
* use the original `training` dataset for `testing` your model
* use the original `testing` dataset for `training` your model

In [3]:
data_path = './MNIST'

data_test   = datasets.MNIST(root = data_path, train= True, download=True, transform= transform)
data_train  = datasets.MNIST(root = data_path, train= False, download=True, transform= transform)


Extracting ./MNIST\MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST\MNIST\raw


Extracting ./MNIST\MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST\MNIST\raw


Extracting ./MNIST\MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST\MNIST\raw


Extracting ./MNIST\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST\MNIST\raw
Processing...


C:\Users\KSY\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


* Note that the number of your `training` data must be 10,000
* Note that the number of your `testing` data must be 60,000


In [4]:
print("the number of your training data (must be 10,000) =", data_train.__len__())
print("the number of your testing data (must be 60,000) =", data_test.__len__())


the number of your training data (must be 10,000) = 10000
the number of your testing data (must be 60,000) = 60000


## Model
* design a neural network architecture with a combination of convolutional layers and fully connected layers
* use any number of feature layers (convolutional layers)
* use any size of convolutional kernel_size
* use any number of classification layers (fully connected layers)
* use any dimension of classification layers
* use any type of activation functions
* one example model of the convolutional neural network is as follows:

In [6]:
class MyModel(nn.Module):

    def __init__(self, num_classes=10, size_kernel=5):

        super(MyModel, self).__init__()

        # *********************************************************************
        # input parameter
        #
        # data size:
        #   mnist   : 28 * 28
        # *********************************************************************
        self.number_class   = num_classes
        self.size_kernel    = size_kernel        
        
        # *********************************************************************
        # feature layer
        # *********************************************************************
        self.conv1          = nn.Conv2d(1, 20, kernel_size=size_kernel, stride=1, padding=int((size_kernel-1)/2), bias=True)
        self.conv2          = nn.Conv2d(20, 50, kernel_size=size_kernel, stride=1, padding=int((size_kernel-1)/2), bias=True)

        self.conv_layer1    = nn.Sequential(self.conv1, nn.MaxPool2d(kernel_size=2), nn.ReLU(True))
        self.conv_layer2    = nn.Sequential(self.conv2, nn.MaxPool2d(kernel_size=2), nn.ReLU(True))

        self.feature        = nn.Sequential(self.conv_layer1, self.conv_layer2)
        
        # *********************************************************************
        # classifier layer
        # *********************************************************************
        self.fc1        = nn.Linear(50*7*7, 50, bias=True)
        self.fc2        = nn.Linear(50, num_classes, bias=True)

        self.fc_layer1  = nn.Sequential(self.fc1, nn.ReLU(True))
        self.fc_layer2  = nn.Sequential(self.fc2, nn.ReLU(True))

        self.classifier = nn.Sequential(self.fc_layer1, self.fc_layer2)
        
        self._initialize_weight()        
        
    def _initialize_weight(self):

        for m in self.modules():
            
            if isinstance(m, nn.Conv2d):
                
                nn.init.xavier_uniform_(m.weight, gain=math.sqrt(2))

                if m.bias is not None:

                    m.bias.data.zero_()

            elif isinstance(m, nn.Linear):

                nn.init.xavier_uniform_(m.weight, gain=math.sqrt(2))
                
                if m.bias is not None:

                    m.bias.data.zero_()

    def forward(self, x):

        x = self.feature(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x


## Loss function
* use any type of loss function
* design the output of the output layer considering your loss function

## Optimization
* use any stochastic gradient descent algorithm for the optimization
* use any size of the mini-batch
* use any optimization algorithm (for example, Momentum, AdaGrad, RMSProp, Adam)
* use any regularization algorithm (for example, Dropout, Weight Decay)
* use any annealing scheme for the learning rate (for example, constant, decay, staircase)

In [7]:
def accuracy(log_pred, y_true):
    y_pred = torch.argmax(log_pred, dim=1)
    return (y_pred == y_true).to(torch.float).mean()


In [ ]:
batch_size = 32
lr=0.5
n_epochs = 100

no_cuda = True
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

train_loader = DataLoader(dataset=data_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=data_test, batch_size=batch_size, shuffle=True)

classifier = MyModel().to(device)
optimizer = torch.optim.SGD(classifier.parameters(), lr)

criterion = nn.NLLLoss()

accuracy_stats = {
    'train': [],
    "test": []
}
loss_stats = {
    'train': [],
    "test": []
}

for epoch in range(n_epochs):
    
    # TRAINING
    train_epoch_loss = 0
    train_epoch_acc = 0
    
    classifier.train()
    
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        
        optimizer.zero_grad()
        
        y_train_pred = classifier(X_train_batch)
        
        train_loss = criterion(y_train_pred, y_train_batch)
        train_acc = accuracy(y_train_pred, y_train_batch)
        
        train_loss.backward()
        optimizer.step()
        
        train_epoch_loss += train_loss.item()
        train_epoch_acc += train_acc.item()
        
    with torch.no_grad():
        test_epoch_loss = 0
        test_epoch_acc = 0
        
        classifier.eval()
        
        for X_test_batch, y_test_batch in test_loader:
            X_test_batch, y_test_batch = X_test_batch.to(device), y_test_batch.to(device)
        
            y_test_pred = classifier(X_test_batch)
            
            test_loss = criterion(y_test_pred, y_test_batch)
            test_acc = accuracy(y_test_pred, y_test_batch)
            
            test_epoch_loss += test_loss.item()
            test_epoch_acc += test_acc.item()
            
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['test'].append(test_epoch_loss/len(test_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['test'].append(test_epoch_acc/len(test_loader))
         
print('done')

In [9]:
print(loss_stats['test'])

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


In [10]:
plt.figure(1,figsize=(9,9))
plt.plot(np.array(range(n_epochs)), loss_stats['train'], c='r', label='train_loss')
plt.plot(np.array(range(n_epochs)), loss_stats['test'], label='test_loss')
plt.legend()
plt.title('loss')
plt.show()

In [11]:
plt.figure(1,figsize=(9,9))
plt.plot(np.array(range(n_epochs)), accuracy_stats['train'], c='r', label='train_loss')
plt.plot(np.array(range(n_epochs)), accuracy_stats['test'], label='test_loss')
plt.legend()
plt.title('accuracy')
plt.show()

In [86]:
final_train_loss = []
final_test_loss = []
final_train_acc = []
final_test_acc = []

In [108]:
final_train_loss.append(loss_stats['train'][-1])
final_test_loss.append(loss_stats['test'][-1])
final_train_acc.append(accuracy_stats['train'][-1])
final_test_acc.append(accuracy_stats['test'][-1])

In [104]:
print('%.6f' % final_train_loss[1])

0.001297


In [115]:
print('mini-batch size = 32 | training accuracy = %.6f'% final_train_acc[0], '| testing accuracy = %.6f'% final_test_acc[0])
print('mini-batch size = 64 | training accuracy = %.6f'% final_train_acc[1], '| testing accuracy = %.6f'% final_test_acc[1])
print('mini-batch size = 128 | training accuracy = %.6f'% final_train_acc[2], '| testing accuracy = %.6f'% final_test_acc[2])

mini-batch size = 32 | training accuracy = 1.000000 | testing accuracy = 0.985124
mini-batch size = 64 | training accuracy = 1.000000 | testing accuracy = 0.982882
mini-batch size = 128 | training accuracy = 0.999650 | testing accuracy = 0.982298


## 1. Plot the training and testing losses over epochs [2pt]

In [36]:
plt.figure(1,figsize=(9,9))
plt.plot(np.array(range(n_epochs)), loss_stats['train'], c='r', label='train_loss')
plt.plot(np.array(range(n_epochs)), loss_stats['test'], label='test_loss')
plt.legend()
plt.title('loss')
plt.show()

## 2. Plot the training and testing accuracies over epochs [2pt]

In [37]:
plt.figure(1,figsize=(9,9))
plt.plot(np.array(range(n_epochs)), accuracy_stats['train'], c='r', label='train_loss')
plt.plot(np.array(range(n_epochs)), accuracy_stats['test'], label='test_loss')
plt.legend()
plt.title('accuracy')
plt.show()

## 3. Print the final training and testing losses at convergence [2pt]
* `*** NOTE ***` The values should be presented up to 5 decimal places (소수점 5째 자리까지 표기하시오)

In [39]:
print('train loss : %.6f' %loss_stats['train'][-1])
print('test loss : %.6f' %loss_stats['test'][-1])

train loss : 0.002180
test loss : 0.190550


## 4. Print the final training and testing accuracies at convergence [20pt]
* `*** NOTE ***` The values should be presented up to 5 decimal places (소수점 5째 자리까지 표기하시오)

In [41]:
print('train accuracy " %.6f' %accuracy_stats['train'][-1])
print('test accuracy " %.6f' %accuracy_stats['test'][-1])

train accuracy " 1.000000
test accuracy " 0.953483


## 5. Print the testing accuracies within the last 10 epochs [5pt]
* `*** NOTE ***` The values should be presented up to 5 decimal places (소수점 5째 자리까지 표기하시오)